In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import matplotlib.pyplot as plt 
#import plotly.express as px 
import tensorflow 
import pickle

from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, accuracy_score

import warnings 
warnings.simplefilter(action='ignore')

Defining directories for each class

In [3]:
train_path = 'plant-disease-recognition-dataset/Train/Train'
validation_path = 'plant-disease-recognition-dataset/Validation/Validation'
test_path = 'plant-disease-recognition-dataset/Test/Test'

# Image Augmentation as a part for prerprocessing

In [4]:
train_batches = ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), classes=["Healthy", "Powdery", "Rust"], batch_size=10) 
validation_batches = ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=validation_path, target_size=(224,224), classes=["Healthy", "Powdery", "Rust"], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(224,224), classes=["Healthy", "Powdery", "Rust"], batch_size=10, shuffle=False)

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [5]:
os.listdir(train_path)

['Healthy', 'Powdery', 'Rust']

# Making use of the pre-defined ResNet50 model

In [6]:
from tensorflow.keras.applications import ResNet50

In [7]:
base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet") # initializer the model

In [8]:
base_model.summary() # architecture of the model.

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [9]:
for layer in base_model.layers:
    layer.trainable = False

In [10]:
base_model = Sequential() # define our own model 
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max')) # add the reset50 model layers
base_model.add(Dense(units=3, activation='sigmoid')) # add a output layer 

In [11]:
base_model.summary() # model architecture with output dense layer 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [12]:
base_model.compile(optimizer = SGD(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Training time for the model was more then other models considering the architecture

In [13]:
reset_his = base_model.fit(x=train_batches, validation_data=validation_batches, epochs=5, verbose=2)

Epoch 1/5
133/133 - 550s - loss: 2.0619 - accuracy: 0.6452 - val_loss: 0.8276 - val_accuracy: 0.8000 - 550s/epoch - 4s/step
Epoch 2/5
133/133 - 523s - loss: 0.4106 - accuracy: 0.8971 - val_loss: 0.4978 - val_accuracy: 0.9167 - 523s/epoch - 4s/step
Epoch 3/5
133/133 - 518s - loss: 0.1773 - accuracy: 0.9561 - val_loss: 0.4434 - val_accuracy: 0.9167 - 518s/epoch - 4s/step
Epoch 4/5
133/133 - 518s - loss: 0.1305 - accuracy: 0.9584 - val_loss: 0.3798 - val_accuracy: 0.9000 - 518s/epoch - 4s/step
Epoch 5/5
133/133 - 540s - loss: 0.0686 - accuracy: 0.9728 - val_loss: 0.3447 - val_accuracy: 0.8833 - 540s/epoch - 4s/step


In [14]:
base_model.save('resnet_50model.h5') # saving the model for future use and possible changes 

In [15]:
resnet_model = tensorflow.keras.models.load_model("./resnet_50model.h5") # loading the model
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
resnet_pred = base_model.predict(test_batches) # predictions using reset50 model
resnet_pred = np.argmax(resnet_pred, axis=1) # rounding of the output values

15/15 [==============================] - 27s 2s/step


In [17]:
resnet_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [18]:
accuracy_score(test_batches.classes, resnet_pred) # schecking the accuracy for the resnet50 model

0.9333333333333333

In [19]:
confusion_matrix(test_batches.classes, resnet_pred)

array([[47,  2,  1],
       [ 4, 45,  1],
       [ 2,  0, 48]], dtype=int64)

### Teste predict

In [28]:
def predict(image_file):
    
    image = ImageDataGenerator(preprocessing_function=tensorflow.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=image_file, target_size=(224,224), classes=["Healthy", "Powdery", "Rust"], batch_size=10) 

    prediction = base_model.predict(image)

    print(np.argmax(prediction, axis=1))

In [29]:
predict('plant-disease-recognition-dataset/Test/Test/86aeb8784b794d58.jpg')

Found 0 images belonging to 3 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0